In [1]:
import itertools
import os
import re
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
def subsets(iterable, minimum=0):
    for i in range(minimum, len(iterable) + 1):
        for x in itertools.combinations(iterable, i):
            yield x

def one_in_another(s1, s2):
    if s1 == s2:
        return True
    if len(s1) > len(s2):
        return one_in_another(s2, s1)
    for i in range(len(s2) + 1):
        for j in range(i):
            if s2.replace(s2[j:i],'') == s1:
                return True
    return False

In [55]:
class Chord(object):
    
    def __init__(self, root=None, intervals=set()):
        self.root = root
        self.intervals = intervals
        if root is not None:
            self.intervals.add(root)
    
    def __str__(self):
        return '(root: ' + str(self.root) + ' intervals: ' + ', '.join([str(x) for x in self.intervals]) + ')'
    
    def __repr__(self):
        return self.__str__()
    
    def __eq__(self, other):
        return self.root == other.root and self.intervals == other.intervals
    
    def transpose(self, half_steps=1):
        if self.root == None:
            return self
        return Chord((self.root + half_steps) % 12, set((i + half_steps) % 12 for i in self.intervals))
    
    def normalize(self):
        if self.root is None:
            return self
        return self.transpose(-1 * self.root)
    
    def add_note(self, note):
        return Chord(self.root, self.intervals.union([note % 12]))
    
    def signature(self):
        return [1 * (i == self.root) for i in range(12)] + [1 * (i in self.intervals) for i in range(12)]
    
    @staticmethod
    def all_chords():
        for x in subsets(range(12), minimum=3):
            temp = set(x)
            for i in temp:
                yield Chord(i, temp)

In [4]:
notes = [s + q for q in 'b#' for s in 'ABCDEFG'] + [s for s in 'ABCDEFG']

In [5]:
# def get_chord_dict():
#     other_chord_dict = []
#     with open('./chordmeanings2.txt') as stuff:
#         for x in stuff.readlines():
#             other_chord_dict += [(x[24:-3].replace('"','').split(', '))[::-1]]
#     other_chord_dict = dict(other_chord_dict)
#     other_chord_dict['NC'] = 'NC'
#     other_chord_dict['CBass'] = 'Bass'
#     other_chord_dict
#     chord_dict = dict()
#     with open('./chordmeanings1.txt') as stuff:
#         for x in stuff.readlines():
#             temp = x.split()
#             try:
#                 chord_dict[other_chord_dict[temp[4]]] = [int(x[-2]) for x in temp[6:-1]]
#             except:
#                 print(temp[4])
#     return chord_dict

def isolate_stem(chord_word):
    if 'Bass' in chord_word:
        return 'Bass'
    if '\\' in chord_word:
        return
    elif '/' in chord_word:
        temp = chord_word.split('/')[0]
        for n in notes:
            if n in temp:
                return temp.replace(n, '')
    else:
        for n in notes:
            if n in chord_word:
                return chord_word.replace(n, '')

def stematize(chord):
    for note in notes:
        if note in chord:
            return stematize(chord.replace(note, str(note_to_num(note))))
    return chord

def note_to_num(note):
    notedict = {'A':9,'B':11,'C':0,'D':2,'E':4,'F':5,'G':7}
    try:
        number = notedict[note[0]]
    except:
        number = 0
    if '#' in note:
        number = (number + 1) % 12
    if 'b' in note:
        number = (number - 1) % 12
    return number

# def num_to_note(n):
#     notedict = {'A':9,'B':11,'C':0,'D':2,'E':4,'F':5,'G':7}

note_cycle = dict()
for note in notes:
    temp = []
    n = note_to_num(note)
    for note2 in notes:
        if (note_to_num(note2) - 1) % 12 == n:
            temp += [note2]
    note_cycle[note] = temp

def parse_chordline(s):
    temp = s.split()
    out = []
    contin = True
    while contin:
        out = []
        contin = False
        for i, x in enumerate(temp):
            if x == '|' or x == '/':
                out += [temp[i - 1]]
                contin = True
            else:
                out += [x]
        temp = out[:]
    return out

def stretch_chords(c):
    if len(c) == 0:
        return c
    if len(c) == 1:
        return c + c + c + c + c + c + c + c
    elif len(c) == 2:
        return c + c + c + c
    elif len(c) == 4:
        return c + c
    elif len(c) == 8:
        return c
    else:
        raise Exception()

def parse_chordlines(chordlines):
    out = sum([stretch_chords(y.split()) for y in (''.join([x[:-1] for x in chordlines])).split('|')], [])
    for i, c in enumerate(out):
        if c == '/':
            out[i] = out[i - 1]
    return out

def get_chords(path):
    with open(path) as leadsheet:
        chordlines = []
        temp = ''
        for line in leadsheet.readlines():
            if '|' in line:
                temp += re.sub("[\(\[].*?[\)\]]", "", line.replace('\n',''))
        return [x.replace('|', '') for x in parse_chordline(temp) if x != ')']

# def get_chords(path):
#     with open(path) as leadsheet:
#         print(path)
#         temp = [x for x in leadsheet.readlines() if '|' in x]
#     return parse_chordlines([x for x in temp if x != ')'])

def process_leadsheets():
    sheet_dict = dict()
    for path in ['./the-imaginary-book-part-1-A-M/', './the-imaginary-book-part-2-N-Z/']:
        for fn in os.listdir(path=path):
            if '.ls' in fn:
                sheet_dict[fn.replace('.ls', '')] = ' '.join(get_chords(path + fn))
    with open('sheets.json', 'w') as file:
        json.dump(sheet_dict, file)

In [6]:
new_chord_defs = ('C.............. same as CM\nCM............. C major (c e g)\nC2............. same as CMadd9\nC5............. C five (c g)\nC6............. same as CM6\nC69............ same as CM69\nC6#11.......... same as CM6#11\nC69#11......... same as CM69#11\nC6b5........... same as CM6#11\nCM13........... C major thirteen (c e g b d a)\nCM13#11........ C major thirteen sharp eleven (c e g b d f# a)\nCmaj13......... same as CM13\nCMaj13......... same as CM13\nCmaj13#11...... same as CM13#11\nCMaj13#11...... same as CM13#11\nCM6............ C major six (c e g a)\nCM6#11......... C major six sharp eleven (c e g a f#)\nCM6b5.......... same as CM6#11\nCM69#11........ C major six nine sharp eleven (c e g a d f#)\nCM69........... C major six nine (c e g a d)\nCM7#11......... C major seven sharp eleven (c e g b f#)\nCM7............ C major seven (c e g b)\nCmaj7.......... same as CM7\nCMaj7.......... same as CM7\nCmaj7#11....... same as CM7#11\nCMaj7#11....... same as CM7#11\nCM7add13....... C major seven add 13 (c e g a b d)\nCM7b5.......... C major seven flat five (c e gb b)\nCM7b6.......... C major seven flat six (c e g ab b)\nCM7b9.......... C major seven flat nine (c e g b db)\nCM9............ C major nine (c e g b d)\nCM9#11......... C major nine sharp eleven (c e g b d f#)\nCmaj9.......... same as CM9\nCMaj9.......... same as CM9\nCmaj9#11....... same as CM9#11\nCMaj9#11....... same as CM9#11\nCM9b5.......... C major nine flat five (c e gb b d)\nCMadd9......... C major add nine (c e g d)\nCMb5........... C major flat five (c e gb)\nCMb6........... C major flat six (c e ab)\nCadd2.......... same as CMadd9\nCadd9.......... same as CMadd9\nCadd9no3....... same as CMsus2\n\nMinor Chords\n\nCm#5........... C minor sharp five (c eb g#)\nCm+............ same as Cm#5\nCm............. C minor (c eb g)\nCm11#5......... C minor eleven sharp five (c eb ab bb d f)\nCm11........... C minor eleven (c eb g bb d f)\nCm11b5......... C minor eleven flat five (c eb bb gb d f)\nCm13........... C minor thirteen (c eb g bb d f a)\nCm6............ C minor six (c eb g a)\nCm69........... C minor six nine (c eb g a d)\nCm7#5.......... C minor seven sharp five (c eb ab bb)\nCm7............ C minor seven (c eb g bb)\nCm7b5.......... C minor seven flat five (c eb gb bb)\nCh............. same as Cm7b5 (h for \"half-diminished\")\nCm9#5.......... C minor nine sharp five (c eb ab bb d)\nCm9............ C minor nine (c eb g bb d)\nCm9b5.......... C minor nine flat five (c eb bb gb d)\nCmM7........... C minor major seven (c eb g b)\nCmM7b6......... C minor major seven flat six (c eb g ab b)\nCmM9........... C minor major nine (c eb g b d)\nCmadd9......... C minor add nine (c eb g d)\nCmb6........... C minor flat six (c eb ab)\nCmb6M7......... C minor flat six major 7 (c eb ab b)\nCmb6b9......... C minor flat six flat nine (c eb ab db)\n\nDiminished Chords\n\nCdim........... C diminished triad (c eb gb)\nCo............. same as Cdim\nCdim7.......... C diminished seventh (c eb gb a)\nCo7............ same as Cdim7\nCoM7........... C diminished major seventh (c eb gb b)\nCo7M7.......... C diminished seventh major seventh (c eb gb a b)\n\nAugmented Chords\n\nCM#5........... C major sharp five (c e g#)\nC+............. same as CM#5\nCaug........... same as CM#5\nC+7............ same as C7#5\nCM#5add9....... C major sharp five add 9 (c e g# d)\nCM7#5.......... C major seven sharp five (c e g# b)\nCM7+........... same as CM7#5\nCM9#5.......... C major nine sharp five (c e g# b d)\nC+add9......... same as CM#5add9\n\nDominant Chords\n\nC7............. C seven (c e g bb)\nC7#5........... C seven sharp five (c e g# bb)\nC7+............ same as C7#5\nCaug7.......... same as C7#5\nC7aug.......... same as C7#5\nC7#5#9......... C seven sharp five sharp nine (c e g# bb d#)\nC7alt.......... same as C7#5#9\nC7b13.......... C seven flat thirteen (c e g bb ab)\nC7b5#9......... same as C7#9#11\nC7b5........... C seven flat five (c e gb bb)\nC7b5b13........ same as C7#11b13\nC7b5b9......... same as C7b9#11\nC7b5b9b13...... same as C7b9#11b13\nC7b6........... C seven flat six (c e g ab bb)\nC7b9#11........ C seven flat nine sharp eleven (c e g bb db f#)\nC7b9#11b13..... C seven flat nine sharp eleven flat thirteen (c e g bb db f# ab)\nC7b9........... C seven flat nine (c e g bb db)\nC7b9b13#11..... C seven flat nine flat thirteen sharp eleven (c e g bb db f# ab)\nC7b9b13........ C seven flat nine flat thirteen (c e g bb db ab)\nC7no5.......... C seven no five (c e bb)\nC7#11.......... C seven sharp eleven (c e g bb f#)\nC7#11b13....... C seven sharp eleven flat thirteen (c e g bb f# ab)\nC7#5b9#11...... C seven sharp five flat nine sharp 11 (c e g# bb db f#)\nC7#5b9......... C seven sharp five flat nine (c e g# bb db)\nC7#9#11........ C seven sharp nine sharp eleven (c e g bb d# f#)\nC7#9#11b13..... C seven sharp nine sharp eleven flat thirteen (c e g bb d# f# ab)\nC7#9........... C seven sharp nine (c e g bb d#)\nC7#9b13........ C seven sharp nine flat thirteen (c e g bb d# ab)\n\nC9............. C nine (c e g bb d)\nC9#5........... C nine sharp five (c e g# bb d)\nC9+............ same as C9#5\nC9#11.......... C nine sharp eleven (c e g bb d f#)\nC9#11b13....... C nine sharp eleven flat thirteen (c e g bb d f# ab)\nC9#5#11........ C nine sharp five sharp eleven (c e g# bb d f#)\nC9b13.......... C nine flat thirteen (c e g bb d ab)\nC9b5........... C nine flat five (c e gb bb d)\nC9b5b13........ same as C9#11b13\nC9no5.......... C nine no five (c e bb d)\n\nC13#11......... C thirteen sharp eleven (c e g bb d f# a)\nC13#9#11....... C thirteen sharp nine sharp eleven (c e g bb d# f# a)\nC13#9.......... C thirteen sharp nine (c e g bb d# a)\nC13............ C thirteen (c e g bb d a)\nC13b5.......... C thirteen flat five (c e gb a bb)\nC13b9#11....... C thirteen flat nine sharp eleven (c e g bb db f# a)\nC13b9.......... C thirteen flat nine (c e g bb db a)\n\nSuspensions\n\nCMsus2......... C major sus two (c d g)\nCMsus4......... C major sus four (c f g)\nCsus2.......... same as CMsus2\nCsus24......... C sus two four (c d f g)\nCsus4.......... same as CMsus4\nCsus4add9...... same as Csus24\nCsusb9......... C sus flat nine (c db f g)\nC4............. C four (c f bb eb)\nCquartal....... same as C4\nC7b9b13sus4.... same as C7sus4b9b13\nC7b9sus........ same as C7susb9\nC7b9sus4....... same as C7sus4b9\nC7b9sus4....... same as C7susb9\nC7sus.......... same as C7sus4\nC7sus4......... C seven sus four (c f g bb)\nC7sus4b9....... C seven sus four flat nine (c f g bb db)\nC7sus4b9b13.... C seven sus four flat nine flat thirteen (c f g bb db ab)\nC7susb9........ C seven sus flat nine (c db f g bb)\nC9sus4......... C nine sus four (c f g bb d)\nC9sus.......... same as C9sus4\nC11............ C eleven (c e g bb d f)\nC13sus......... same as C13sus4\nC13sus4........ C thirteen sus four (c f g bb d a)\n\nMiscellaneous\n\nCBlues......... C Blues (c eb f gb g bb) (Use upper case to avoid confusion with Cb = C flat)\nCBass.......... C Bass (c) (Use upper case to avoid confusion with Cb = C flat)\n')
chord_defs = [[y for y in x.split('.') if y!=''] for x in new_chord_defs.split('\n') if '.' in x]
equivs = [[x[0][1:], x[1].replace(' same as ', '').replace(' (h for "half-diminished")', '')[1:]] for x in chord_defs if ' same as ' in x[1]]
equivs += [['mMaj7','mM7'], ['h7','m7b5'], ['mb5', 'dim']]

def make_into_intervals(chord_def):
    temp = re.findall('\(([^\)]+)\)', chord_def[1])[0]
    temp = [chord_def[0][1:], [note_to_num(x.capitalize()) for x in temp.split()]]
    return [temp[0], [1 * (i in temp[1]) for i in range(12)]]

equiv_dict = dict([make_into_intervals(x) for x in chord_defs if ' same as ' not in x[1]])
# print(equivs)
equiv_dict = {**equiv_dict, **dict([[x[0], equiv_dict[x[1]]] for x in equivs])}
equiv_dict['NC'] = [0] * 12

In [7]:
def process_chord(chord):
    if 'NC' in chord:
        return Chord()
    if 'Bass' in chord:
        for note in notes:
            if chord[:len(note)] == note:
                break
        return Chord(note_to_num(note), {note_to_num(note)})
    if '\\' in chord:
        [poly, rest] = chord.split('\\')
        poly = process_chord(poly)
    else:
        poly = None
        rest = chord
    if '/' in chord:
        [shape, root] = rest.split('/')
    else:
        shape = rest
        root = None
    for note in notes:
        if shape[:len(note)] == note:
            break
    stem = shape.replace(note, '')
    chord_obj = Chord(0, set([i for i in range(12) if equiv_dict[stem][i]]))
    chord_obj = chord_obj.transpose(note_to_num(note))
    if root is not None:
        chord_obj.root = note_to_num(root)
    if poly is not None:
        for n in poly.intervals:
            chord_obj = chord_obj.add_note(n)
    return chord_obj

def process_leadsheet(sheet):
    return [process_chord(x) for x in sheet.split()]

def all_transpositions(sheet):
    temp = process_leadsheet(sheet)
    return [[chord.transpose(i) for chord in temp] for i in range(12)]

def transpose_note(note):
    return note_cycle[note][-1]

def transpose_chord_symbol(chord, interval=1):
    if interval == 0:
        return chord
    elif interval == 1:
        if 'NC' in chord:
            return 'NC'
        if 'Bass' in chord:
            for note in notes:
                if chord[:len(note)] == note:
                    break
            return transpose_note(note) + 'Bass'
        if '\\' in chord:
            return '\\'.join([transpose_chord_symbol(x) for x in chord.split('\\')])
        if '/' in chord:
            return '/'.join([transpose_chord_symbol(x) for x in chord.split('/')])
        for note in notes:
            if chord[:len(note)] == note:
                break
        stem = chord.replace(note, '')
        return transpose_note(note) + stem
    elif 1 < interval < 12:
        return transpose_chord_symbol(transpose_chord_symbol(chord, interval=interval - 1))
    else:
        return transpose_chord_symbol(chord, interval=(interval % 12))

In [42]:
process_leadsheets()

In [8]:
with open('sheets.json') as file:
    sheet_dict = json.load(file)

In [ ]:
every_sheet = []
for sheet in sheet_dict.values():
    every_sheet += all_transpositions(sheet)

In [9]:
chords = set()
for key in sheet_dict:
    temp = sheet_dict[key].split()
    for chord in temp:
        chords.add(chord)
chords

{'Abm/G',
 'D/G',
 'D9sus4',
 'BbM7',
 'Db9+',
 'Bbm',
 'EM7/B',
 'Eb7#11/G',
 'Gb7alt',
 'AbM7/C',
 'B/Db',
 'G#7#5b9',
 'B\\F7',
 'Eb6',
 'C#m7',
 'Dmb6',
 'Bbm6/F',
 'C#m7/E',
 'F#69',
 'A7#5b9',
 'CM7/D',
 'Bm7/A',
 'G7b13',
 'G7b5/Db',
 'F9/Eb',
 'Em',
 'Bdim',
 'G#m7',
 'Gsus24',
 'B7b9sus4',
 'A#dim',
 'F7b9',
 'Ebm7/Db',
 'Gb/Db',
 'Bsus4',
 'AM7/C#',
 'DM7b5',
 'CM69',
 'Go7/B',
 'Eb7#9',
 'BbM7#11',
 'AbmMaj7',
 'FM6',
 'Gm7b5',
 'FM9',
 'Eb7#5#9/G',
 'Ab7b9b13',
 'Bbadd9',
 'Abm7/Bb',
 'Bb13/E',
 'Em9',
 'Fm7/A#',
 'F#h7',
 'E7susb9',
 'EmM7',
 'G9#11',
 'C/B',
 'F#9b5',
 'Gm6/E',
 'Bb11',
 'Fm7/G#',
 'E69',
 'D+7',
 'Bo7/Eb',
 'A11',
 'G13#11',
 'BbM7/A',
 'Ab7/Gb',
 'G6/B',
 'E+',
 'Gb9+',
 'G7sus',
 'C/D',
 'G#7alt',
 'C#7alt',
 'Ab7/Bb',
 'Db/C',
 'Eb+/Bb',
 'C#7b5',
 'G/C',
 'Bbm7/Db',
 'A7/G',
 'Fo7/C',
 'Gbm11',
 'C9/G',
 'D+',
 'Gb/C',
 'F11',
 'Ao7',
 'Do7',
 'D13sus',
 'Eb/Ab',
 'Bb6/D',
 'Bm7/F#',
 'Aadd9',
 'EM7#5',
 'G#m11',
 'E7+',
 'Bb7+',
 'Gbo',
 'G9+',
 'F9

In [ ]:
chord_formats = set()
for x in chords:
    chord_formats.add(isolate_stem(x))
chord_formats

In [10]:
all_transpositions(sheet_dict['Dexterity'])

[[(root: 10 intervals: 9, 10, 2, 5),
  (root: 10 intervals: 9, 10, 2, 5),
  (root: 0 intervals: 0, 10, 3, 7),
  (root: 5 intervals: 0, 9, 3, 5),
  (root: 5 intervals: 0, 9, 3, 5),
  (root: 10 intervals: 9, 10, 2, 5),
  (root: 10 intervals: 9, 10, 2, 5),
  (root: 0 intervals: 0, 10, 3, 7),
  (root: 5 intervals: 0, 9, 3, 5),
  (root: 5 intervals: 0, 9, 3, 5),
  (root: 10 intervals: 9, 10, 2, 5),
  (root: 10 intervals: 8, 10, 2, 5),
  (root: 10 intervals: 8, 10, 2, 5),
  (root: 3 intervals: 0, 10, 3, 7),
  (root: 8 intervals: 8, 0, 3, 6),
  (root: 8 intervals: 8, 0, 3, 6),
  (root: 2 intervals: 0, 9, 2, 5),
  (root: 7 intervals: 2, 11, 5, 7),
  (root: 7 intervals: 2, 11, 5, 7),
  (root: 0 intervals: 0, 10, 3, 7),
  (root: 5 intervals: 0, 9, 3, 5),
  (root: 5 intervals: 0, 9, 3, 5),
  (root: 10 intervals: 9, 10, 2, 5),
  (root: 10 intervals: 9, 10, 2, 5),
  (root: 0 intervals: 0, 10, 3, 7),
  (root: 5 intervals: 0, 9, 3, 5),
  (root: 5 intervals: 0, 9, 3, 5),
  (root: 10 intervals: 9, 10, 

In [ ]:
training_data = [[x.signature() for x in sheet] for sheet in every_sheet]

In [56]:
chord_objs = dict()
for chord in chords:
    temp = process_chord(chord)
    name = chord
    for _ in range(12):
        name = transpose_chord_symbol(name)
        temp = temp.transpose()
        chord_objs[name] = temp
chord_objs

{'Am/G#': (root: 8 intervals: 0, 9, 4, 8),
 'A#m/A': (root: 9 intervals: 1, 10, 5, 9),
 'Bm/A#': (root: 10 intervals: 10, 2, 11, 6),
 'Cm/B': (root: 11 intervals: 3, 0, 11, 7),
 'C#m/C': (root: 0 intervals: 0, 1, 4, 8),
 'Dm/C#': (root: 1 intervals: 1, 2, 5, 9),
 'D#m/D': (root: 2 intervals: 10, 2, 3, 6),
 'Em/D#': (root: 3 intervals: 3, 11, 4, 7),
 'Fm/E': (root: 4 intervals: 0, 8, 4, 5),
 'F#m/F': (root: 5 intervals: 1, 5, 9, 6),
 'Gm/F#': (root: 6 intervals: 2, 10, 6, 7),
 'G#m/G': (root: 7 intervals: 11, 8, 3, 7),
 'D#/G#': (root: 8 intervals: 8, 10, 3, 7),
 'E/A': (root: 9 intervals: 8, 9, 11, 4),
 'F/A#': (root: 10 intervals: 0, 9, 10, 5),
 'F#/B': (root: 11 intervals: 1, 10, 11, 6),
 'G/C': (root: 0 intervals: 0, 2, 11, 7),
 'G#/C#': (root: 1 intervals: 0, 1, 3, 8),
 'A/D': (root: 2 intervals: 1, 2, 4, 9),
 'A#/D#': (root: 3 intervals: 10, 2, 3, 5),
 'B/E': (root: 4 intervals: 3, 11, 4, 6),
 'C/F': (root: 5 intervals: 0, 4, 5, 7),
 'C#/F#': (root: 6 intervals: 8, 1, 5, 6),
 'D/G

In [211]:
eclasses = []
for value in chord_objs.values():
    if value.root == 0:
        temp = []
        for key in chord_objs.keys():
            if chord_objs[key] == value:
                temp += [key]
        if temp not in eclasses:
            eclasses += [temp]

            
decisions = [x for x in eclasses if len(x) > 1]

out = []
for d in decisions:
    cont = False
    for x in d:
        if 'maj' in x:
            cont = True
            break
    if cont:
        temp = [x for x in d if 'maj' not in x]
        if len(temp) > 0:
            out += [temp]
        else:
            out += [d]
    else:
        out += [d]

out2 = []
for d in out:
    cont = False
    for x in d:
        if 'sus' in x:
            cont = True
            break
    if cont:
        temp = [x for x in d if 'sus' not in x or 'sus4' in x or 'sus2' in x]
        if len(temp) > 0:
            out2 += [temp]
        else:
            out2 += [d]
    else:
        out2 += [d]

out3 = []
for d in out2:
    cont = False
    for x in d:
        if '\\' in x:
            cont = True
            break
    if cont:
        temp = [x for x in d if '\\' not in x]
        if len(temp) > 0:
            out3 += [temp]
        else:
            out3 += [d]
    else:
        out3 += [d]

out4 = []
for d in out3:
    cont = False
    for x in d:
        if '6' in x:
            cont = True
            break
    if cont:
        temp = [x for x in d if '6' not in x]
        if len(temp) > 0:
            out4 += [temp]
        else:
            out4 += [d]
    else:
        out4 += [d]

out5 = []
for d in out4:
    cont = False
    for x in d:
        if '/' in x:
            cont = True
            break
    if cont:
        temp = [x for x in d if '/' not in x]
        if len(temp) > 0:
            out5 += [temp]
        else:
            out5 += [d]
    else:
        out5 += [d]

out6 = []
for x in out5:
    if len(x) == 2 and one_in_another(*x):
        out6 += [[min(x,key=len)]]
    else:
        out6 += [x]
        
decided = [x[0] for x in out6 + [y for y in eclasses if len(y) > 0]] + ['NC']
canonical_chords = []
for d in decided:
    for i in range(12):
        temp = transpose_chord_symbol(d, i)
        if temp not in canonical_chords:
            canonical_chords += [temp]